In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
df = pd.read_csv('email.csv').dropna(subset= ['content'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Котя\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Котя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df['tokens'] = df.one_line.apply(lambda x: str(x).split(" "))

In [8]:
def lemmatize(tokens):
    result = [get_lemma(token) for token in tokens if len(token)>2 and token not in en_stop]
    return result  

In [12]:
data = pd.DataFrame()
data['lemmatized_tokens'] = [lemmatize(tokens) for tokens in df['tokens'][0:2000]]

In [15]:
df.to_csv("lemmatized.csv")

In [13]:
from gensim.models.ldamodel import LdaModel

In [14]:
from gensim import corpora
dictionary = corpora.Dictionary(data['lemmatized_tokens'])
corpus = [dictionary.doc2bow(text) for text in data['lemmatized_tokens']]
import pickle
pickle.dump(corpus, open('corpus2000.pkl', 'wb'))
dictionary.save('dictionary2000.gensim')

In [15]:
ldamodel = LdaModel(corpus, num_topics = 20, id2word=dictionary, passes=15)
ldamodel.save('model100.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.011*"The" + 0.011*"market" + 0.008*"ECT" + 0.006*"Enron"')
(1, '0.017*"Exchange" + 0.015*"contract" + 0.015*"shall" + 0.014*"The"')
(2, '0.024*"month" + 0.019*"Division" + 0.018*"COMEX" + 0.018*"Members"')
(3, '0.030*"NYMEX" + 0.024*"Futures" + 0.023*"Margin" + 0.020*"Member"')
(4, '0.032*"Price" + 0.032*"Save" + 0.031*"Reg" + 0.024*"For"')
(5, '0.035*"Thanks" + 0.028*"deal" + 0.023*"ECT" + 0.021*"Subject"')
(6, '0.012*"The" + 0.009*"COMMITTEE" + 0.007*"message" + 0.007*"Board"')
(7, '0.031*"Time" + 0.022*"Event" + 0.021*"position" + 0.019*"All"')
(8, '0.033*"Copper" + 0.011*"BHP" + 0.010*"COMEX" + 0.008*"Approved"')
(9, '0.021*"margin" + 0.016*"order" + 0.013*"Clearing" + 0.012*"customer"')
(10, '0.009*"say" + 0.007*"would" + 0.005*"get" + 0.005*"Enron"')
(11, '0.032*"price" + 0.025*"limit" + 0.023*"contract" + 0.022*"NYMEX"')
(12, '0.099*"IMAGE" + 0.030*"ClearStation" + 0.021*"Securities" + 0.019*"Bank"')
(13, '0.011*"The" + 0.011*"roundtrip" + 0.010*"New" + 0.008*"York"')
(14

In [8]:
import pyLDAvis.gensim 
from gensim.models.ldamodel import LdaModel
import pickle

corpus = pickle.load(open('corpus2000.pkl', 'rb'))
lda = LdaModel.load('model100.gensim')


In [11]:
from gensim.corpora import Dictionary
dictionary = Dictionary.load('dictionary2000.gensim')

In [13]:
lda_display10 = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)